In [77]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import pandas as pd
import re

from env import github_token, github_username

In [46]:
ubuntu_df = pd.read_json('ubuntu_data.json')

In [73]:
soup = BeautifulSoup(ubuntu_df.readme_contents[1], 'html.parser')

In [87]:
pd.Series([word for word in re.sub(r'[^\w]', ' ', soup.text).split()]).value_counts()

the         54
to          37
box         18
be          16
for         16
            ..
commits      1
requires     1
targets      1
detritus     1
specify      1
Length: 416, dtype: int64

In [82]:
soup.text

"# Packer templates for Ubuntu written in legacy JSON\n\n### Overview\n\nThis repository contains [Packer](https://packer.io/) templates for creating Ubuntu Vagrant boxes written in legacy JSON.\n\n## Current Boxes\n\nWe no longer provide pre-built binaries for these templates.\n\n## Building the Vagrant boxes with Packer\n\nTo build all the boxes, you will need [VirtualBox](https://www.virtualbox.org/wiki/Downloads), \n[VMware Fusion](https://www.vmware.com/products/fusion)/[VMware Workstation](https://www.vmware.com/products/workstation) and\n[Parallels](http://www.parallels.com/products/desktop/whats-new/) installed.\n\nParallels requires that the\n[Parallels Virtualization SDK for Mac](http://www.parallels.com/downloads/desktop)\nbe installed as an additional prerequisite.\n\nWe make use of JSON files containing user variables to build specific versions of Ubuntu.\nYou tell `packer` to use a specific user variable file via the `-var-file=` command line\noption.  This will override 

In [88]:
([word for word in re.sub(r'[^\w]', ' ', soup.text).split()])

['Packer',
 'templates',
 'for',
 'Ubuntu',
 'written',
 'in',
 'legacy',
 'JSON',
 'Overview',
 'This',
 'repository',
 'contains',
 'Packer',
 'https',
 'packer',
 'io',
 'templates',
 'for',
 'creating',
 'Ubuntu',
 'Vagrant',
 'boxes',
 'written',
 'in',
 'legacy',
 'JSON',
 'Current',
 'Boxes',
 'We',
 'no',
 'longer',
 'provide',
 'pre',
 'built',
 'binaries',
 'for',
 'these',
 'templates',
 'Building',
 'the',
 'Vagrant',
 'boxes',
 'with',
 'Packer',
 'To',
 'build',
 'all',
 'the',
 'boxes',
 'you',
 'will',
 'need',
 'VirtualBox',
 'https',
 'www',
 'virtualbox',
 'org',
 'wiki',
 'Downloads',
 'VMware',
 'Fusion',
 'https',
 'www',
 'vmware',
 'com',
 'products',
 'fusion',
 'VMware',
 'Workstation',
 'https',
 'www',
 'vmware',
 'com',
 'products',
 'workstation',
 'and',
 'Parallels',
 'http',
 'www',
 'parallels',
 'com',
 'products',
 'desktop',
 'whats',
 'new',
 'installed',
 'Parallels',
 'requires',
 'that',
 'the',
 'Parallels',
 'Virtualization',
 'SDK',
 'for',
 